In [2]:
!pip3 install fuzzywuzzy

In [4]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.1 MB/s eta 0:00:00


In [8]:
import requests
from bs4 import BeautifulSoup

paronyms = []


# взятие паронимов с открытого ресурса
home_url = 'https://ru-paronym.ru'
ru_alphabet = [chr(i) for i in range(ord('А'), ord('А') + 32)]
no_words = ['Ё', 'Й', 'Щ', 'Ъ', 'Ы', 'Ь']


for letter in ru_alphabet:
    if letter in no_words:
        continue
    url = home_url + '/' + letter
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    arr = soup.find_all('a', href=True)[2:-27]  # Откидываем первые 2 и последние 27 элемента, которые есть на каждой странице
    for elem in arr:
        webpage = home_url + elem['href']
        words = (elem.string).split(' — ')
        paronyms.append(((words[0], words[1]), 1))  # пары слов - паронимы
        paronyms.append(((words[1], words[0]), 1))  # пара в обратном порядке





In [9]:
# база данных с кортежами с меткой (1) - паронимы
print(paronyms)

[(('абонемент', 'абонент'), 1), (('абонент', 'абонемент'), 1), (('абстрактность', 'абстракция'), 1), (('абстракция', 'абстрактность'), 1), (('авантюрин', 'авантюрист'), 1), (('авантюрист', 'авантюрин'), 1), (('автобиографический', 'автобиографичный'), 1), (('автобиографичный', 'автобиографический'), 1), (('автоматический', 'автоматичный'), 1), (('автоматичный', 'автоматический'), 1), (('автономия', 'автономность'), 1), (('автономность', 'автономия'), 1), (('авторитарный', 'авторитетный'), 1), (('авторитетный', 'авторитарный'), 1), (('адресант', 'адресат'), 1), (('адресат', 'адресант'), 1), (('азарт', 'азартность'), 1), (('азартность', 'азарт'), 1), (('академический', 'академичный'), 1), (('академичный', 'академический'), 1), (('аккордный', 'аккордовый'), 1), (('аккордовый', 'аккордный'), 1), (('акселерат', 'акселератор'), 1), (('акселератор', 'акселерат'), 1), (('активировать', 'активизировать'), 1), (('активизировать', 'активировать'), 1), (('акулий', 'акуловый'), 1), (('акуловый', 'а

In [10]:
import random


# создание пар слов не паронимов путем перемешивания уже имеющихся слов
def generate_non_paronyms(paronyms, seed = None):
    all_words = set()

    # Создаем множество всех слов из датасета паронимов
    for pair, _ in paronyms:
        all_words.update(pair)

    non_paronyms = []

    if seed:
      random.seed(seed)

    # Случайные пары из множества всех слов
    for _ in range(len(paronyms)):
        word1, word2 = random.choices(list(all_words), k=2)
        if ((word1, word2), 1) not in paronyms:
          non_paronyms.append(((word1, word2), 0))  # Метка 0, так как это непаронимы

    return non_paronyms

non_paronyms = generate_non_paronyms(paronyms, 42)


In [11]:
dataset = paronyms + non_paronyms
random.shuffle(dataset) # перемешивание датасета на всякий случай
print(dataset[:10])

[(('разливчатый', 'безличный'), 0), (('целый', 'цельный'), 1), (('аргонавт', 'гражданственный'), 0), (('начертать', 'признательный'), 0), (('сыскать', 'снискать'), 1), (('начертать', 'начертить'), 1), (('подпятник', 'абонент'), 0), (('личностный', 'личный'), 1), (('чеканный', 'чеканочный'), 1), (('крючкотворский', 'избирательский'), 0)]


In [ ]:
import pandas as pd

data = pd.DataFrame(dataset, columns=['word_pair', 'label'])

In [19]:
from fuzzywuzzy import fuzz
import Levenshtein

def levenshtein_distance(dataset): # вычисление расстояния левенштейна и сходства слов
    distances = []

    for (word1, word2), _ in dataset:
        lev_dist = Levenshtein.distance(word1, word2)
        lev_ratio = fuzz.ratio(word1, word2)
        distances.append((word1, word2, lev_dist, lev_ratio))

    return distances

In [20]:
lev = levenshtein_distance(dataset)
lev

[('разливчатый', 'безличный', 5, 60),
 ('целый', 'цельный', 2, 83),
 ('аргонавт', 'гражданственный', 12, 26),
 ('начертать', 'признательный', 10, 36),
 ('сыскать', 'снискать', 2, 80),
 ('начертать', 'начертить', 1, 89),
 ('подпятник', 'абонент', 8, 25),
 ('личностный', 'личный', 4, 75),
 ('чеканный', 'чеканочный', 2, 89),
 ('крючкотворский', 'избирательский', 9, 43),
 ('хлеборобный', 'хлеборобский', 3, 78),
 ('керосиновый', 'земной', 7, 47),
 ('заплатить', 'зеленеть', 5, 47),
 ('атавистичный', 'вклад', 11, 12),
 ('игристый', 'игривый', 2, 80),
 ('возмутительный', 'инновация', 14, 17),
 ('светить', 'светлить', 1, 93),
 ('автоматичный', 'автоматический', 4, 77),
 ('цикличный', 'циклический', 4, 70),
 ('накрапывать', 'накрапливать', 2, 87),
 ('заживлять', 'масленый', 8, 24),
 ('надувательский', 'надувательный', 3, 81),
 ('земляной', 'земной', 2, 86),
 ('критичный', 'нафталин', 8, 24),
 ('информативный', 'демократичность', 10, 43),
 ('оделить', 'всход', 7, 33),
 ('анархический', 'неудачлив